# 💬 AI Chatbot for Mental Health Support
---
This project demonstrates a simple AI-powered chatbot that provides empathetic, supportive responses using Natural Language Processing (NLP).

## 📦 Install Required Libraries

In [ ]:

!pip install transformers torch streamlit nltk


## 🧠 Import Libraries

In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import nltk
nltk.download('punkt')


## 🤖 Load Pre-trained Conversational Model

In [ ]:

# Load DialoGPT-small from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-small")


## 💬 Define Chat Function

In [ ]:

def preprocess_input(text):
    negative_words = ["sad", "depressed", "angry", "anxious", "hopeless", "lonely"]
    if any(word in text.lower() for word in negative_words):
        return text + " Please respond with empathy and kindness."
    return text

def chat_with_bot():
    print("Chatbot: Hello, I'm here to listen. How are you feeling today?")
    chat_history_ids = None
    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit", "bye"]:
            print("Chatbot: Take care. You are doing your best 💛")
            break
        new_input_ids = tokenizer.encode(preprocess_input(user_input) + tokenizer.eos_token, return_tensors='pt')
        bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids
        chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
        response = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
        print(f"Chatbot: {response}")
        
# Run this function to start the chatbot
# chat_with_bot()


## 🌐 Optional: Create Streamlit Web App

In [ ]:

# Save this code in a separate file (app.py) to run with: streamlit run app.py

import streamlit as st

st.title("💬 AI Mental Health Support Chatbot")
st.write("I’m here to listen and chat with you in a safe, supportive space.")

if "history" not in st.session_state:
    st.session_state["history"] = None

user_input = st.text_input("You:", "")

if st.button("Send") and user_input:
    new_input_ids = tokenizer.encode(preprocess_input(user_input) + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = torch.cat([st.session_state["history"], new_input_ids], dim=-1) if st.session_state["history"] is not None else new_input_ids
    st.session_state["history"] = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    response = tokenizer.decode(st.session_state["history"][:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    st.text_area("Chatbot:", value=response, height=150)
